In [ ]:
import os

import h5py
import zarr

In [ ]:
path = "/home/projects/qrage/sessions"

In [ ]:
sessions = os.listdir(path)

In [ ]:
sessions.sort()

In [ ]:
for session in sessions:
    print(session)

    if os.path.isfile(
        f"{path}/{session}/preproc/mz_me_mpnrage3d_grappa.h5"
    ) and not os.path.isdir(f"{path}/{session}/preproc.zarr"):

        print(f"Processing {session}")

        store = zarr.storage.LocalStore(
            f"{path}/{session}/preproc.zarr/preproc.zarr",
        )

        root = zarr.create_group(
            store=store,
            overwrite=True,
        )

        try:
            with h5py.File(
                f"{path}/{session}/preproc/mz_me_mpnrage3d_grappa.h5",
                mode="r",
                libver="latest",
            ) as hf:
                for key in hf.keys():
                    print(key, hf[key].shape, hf[key].chunks, hf[key].dtype)
                    array = root.create_array(
                        name=key,
                        shape=hf[key].shape,
                        chunks=hf[key].chunks,
                        dtype=hf[key].dtype,
                        overwrite=True,
                    )
                    array[:] = hf[key][:]

            os.remove(f"{path}/{session}/preproc/mz_me_mpnrage3d_grappa.h5")

        except:
            print(f"{session} failed.")